In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [2]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class3.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [3]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
new_grades_df = grades
for grade in new_grades_df: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " does not contain exactly 26 values:")
                print(grade)
                new_grades_df.remove(grade)
            id = value[0]
            id_length = len(id)
            if id_length != 9:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " is invalid")
                new_grades_df.remove(grade)

            

values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Total valid lines of data: 100
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
Calculate score!!!

In [21]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
answers_checked = []
for grade in grades:
        value = grade.split(",")
        values = []
        if len(value) > 26:
                continue
        elif len(value) < 25:
                continue
        else:
                answer_table = pd.DataFrame(value)
                answer_table = answer_table.iloc[1: , :]
                answer_table = answer_table.reset_index(drop=True)
                cVal = answer_table.iloc[24][0]
                cValSplit = cVal.split("\n")
                c = cValSplit[0]
                answer_table = answer_table.replace([cVal],c)
                conditions  = [answer_table[0] == answer_key_table[0], answer_table[0] == "" , answer_table[0] != answer_key_table[0]]
                choices     = [ "True", '',answer_key_table[0]]
    
                answer_table["checkAnswer"] = np.select(conditions, choices, default=np.nan)
                #answer_table['checkAnswer'] = np.where(answer_table[0] == answer_key_table[0], 'True', answer_key_table[0])
                #answer_table['checkAnswer'] = np.where(answer_table[0] == "" & answer_table[0] != answer_key_table[0] , "", answer_key_table[0])

                #num_correct = answer_table['checkAnswer'].value_counts()['True']
                answers_checked.append(answer_table)
                print(answer_table)


                        

answer_pd = pd.concat(answers_checked)
answer_pd.columns = ['studentAnswer', 'checkAnswer']
answer_pd['question'] = answer_pd.index

#Find out the question students most answer incorrectly and how many time the question is incorrect.
answers_incorrect_pd = answer_pd[answer_pd['checkAnswer'] != "True"]
answers_incorrect_pd = answers_incorrect_pd[answers_incorrect_pd['studentAnswer'] != ""]
answer_incorrect_count = answers_incorrect_pd.groupby('question')['question'].count()
question_most_incorrect = answer_incorrect_count.idxmax()
num_answer_inccorect = answer_incorrect_count.max()
print("Question that most people answer incorrectly:", question_most_incorrect)
print("Question", question_most_incorrect, "is answered incorrectly", num_answer_inccorect, "times")
print("----------------------")

#Find out the question students most leave blank and how many time the question is leave blank.
answers_blank_pd = answer_pd[answer_pd['studentAnswer'] == ""]
answers_blank_count = answers_blank_pd.groupby('question')['question'].count()
question_most_leave_blank = answers_blank_count.idxmax()
num_answer_blank = answers_blank_count.max()
print("Question that most people skip:", question_most_leave_blank)
print("Question", question_most_leave_blank, "is skipped", num_answer_blank, "times")






    0 checkAnswer
0   B        True
1                
2   D        True
3   D        True
4   C        True
5   B        True
6   D        True
7   A        True
8   C        True
9   C        True
10  D        True
11  B        True
12  A        True
13  B        True
14  D           A
15  C        True
16  B        True
17  D        True
18  A        True
19               
20               
21  A        True
22  B        True
23  D        True
24  D        True
    0 checkAnswer
0   B        True
1                
2   D        True
3   C           D
4   C        True
5   B        True
6   D        True
7   D           A
8   C        True
9   C        True
10  D        True
11               
12  A        True
13  B        True
14  D           A
15  C        True
16               
17  D        True
18  A        True
19  C        True
20               
21  D           A
22  B        True
23  D        True
24  C           D
    0 checkAnswer
0   B        True
1   A        True
2   D     

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'